In [1]:
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

sia = SentimentIntensityAnalyzer()

In [ ]:
# Updating Vader Lexicon with External Lexicon
# import csv
# import pandas as pd

# # stock market lexicon
# stock_lex = pd.read_csv('lexicon_data/stock_lex.csv')
# stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
# stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
# stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
# stock_lex_scaled = {}
# for k, v in stock_lex.items():
#     if v > 0:
#         stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
#     else:
#         stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

# # # Loughran and McDonald
# # positive = []
# # with open('lexicon_data/lm_positive.csv', 'r') as f:
# #     reader = csv.reader(f)
# #     for row in reader:
# #         positive.append(row[0].strip())
    
# # negative = []
# # with open('lexicon_data/lm_negative.csv', 'r') as f:
# #     reader = csv.reader(f)
# #     for row in reader:
# #         entry = row[0].strip().split(" ")
# #         if len(entry) > 1:
# #             negative.extend(entry)
# #         else:
# #             negative.append(entry[0])

# final_lex = {}
# # final_lex.update({word:2.0 for word in positive})
# # final_lex.update({word:-2.0 for word in negative})
# final_lex.update(stock_lex_scaled)
# final_lex.update(sia.lexicon)
# sia.lexicon = final_lex

In [ ]:
# Scraping FB posts from Business Times
# from bs4 import BeautifulSoup
# from datetime import datetime, timedelta
# import time
# import pprint

# date_sentiments = {}

# for i in range(1,11):
#     page = urlopen('https://www.businesstimes.com.sg/search/facebook?page='+str(i)).read()
#     soup = BeautifulSoup(page, features="html.parser")
#     posts = soup.findAll("div", {"class": "media-body"})
#     for post in posts:
#         time.sleep(1)
#         url = post.a['href']
#         date = post.time.text
#         print(date, url)
#         try:
#             link_page = urlopen(url).read()
#         except:
#             url = url[:-2]
#             link_page = urlopen(url).read()
#         link_soup = BeautifulSoup(link_page)
#         sentences = link_soup.findAll("p")
#         passage = ""
#         for sentence in sentences:
#             passage += sentence.text
#         sentiment = sia.polarity_scores(passage)['compound']
#         date_sentiments.setdefault(date, []).append(sentiment)

# date_sentiment = {}

# for k,v in date_sentiments.items():
#     date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

# earliest_date = min(date_sentiment.keys())

# print(date_sentiment)

In [ ]:
# # Algorithm

# from __future__ import (absolute_import, division, print_function,
#                             unicode_literals)

# # %matplotlib inline
# # import warnings
# # warnings.filterwarnings('ignore')

# import backtrader as bt
# import backtrader.indicators as btind
# import datetime
# import os.path
# import sys

# class Sentiment(bt.Indicator):
#     lines = ('sentiment',)
#     plotinfo = dict(
#         plotymargin=0.15,
#         plothlines=[0],
#         plotyticks=[1.0, 0, -1.0])
    
#     def next(self):
#         self.date = self.data.datetime
#         date = bt.num2date(self.date[0]).date()
#         prev_sentiment = self.sentiment
#         if date in date_sentiment:
#             self.sentiment = date_sentiment[date]
#         self.lines.sentiment[0] = self.sentiment


# class SentimentStrat(bt.Strategy):
#     params = (
#         ('period', 15),
#         ('printlog', True),
#     )

#     def log(self, txt, dt=None, doprint=False):
#         ''' Logging function for this strategy'''
#         if self.params.printlog or doprint:
#             dt = dt or self.datas[0].datetime.date(0)
#             print('%s, %s' % (dt.isoformat(), txt))

#     def __init__(self):
#         # Keep a reference to the "close" line in the data[0] dataseries
#         self.dataclose = self.datas[0].close
#         # Keep track of pending orders
#         self.order = None
#         self.buyprice = None
#         self.buycomm = None
#         self.sma = bt.indicators.SimpleMovingAverage(
#             self.datas[0], period=self.params.period)
#         self.date = self.data.datetime
#         self.sentiment = None
#         Sentiment(self.data)
        
#     def notify_order(self, order):
#         if order.status in [order.Submitted, order.Accepted]:
#             # Buy/Sell order submitted/accepted to/by broker - Nothing to do
#             return
        
#         # Check if an order has been completed
#         # Attention: broker could reject order if not enough cash
#         if order.status in [order.Completed]:
#             if order.isbuy():
#                 self.log(
#                     'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
#                     (order.executed.price,
#                      order.executed.value,
#                      order.executed.comm))
#                 self.buyprice = order.executed.price
#                 self.buycomm = order.executed.comm
#             else:  # Sell
#                 self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
#                          (order.executed.price,
#                           order.executed.value,
#                           order.executed.comm))
                
#             self.bar_executed = len(self)     
            
#         elif order.status in [order.Canceled, order.Margin, order.Rejected]:
#             self.log('Order Canceled/Margin/Rejected')
            
#         # Write down: no pending order
#         self.order = None
        
#     def notify_trade(self, trade):
#         if not trade.isclosed:
#             return

#         self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
#                  (trade.pnl, trade.pnlcomm))
    
#     ### Main Strat ###
#     def next(self):
#         # log closing price of the series from the reference
#         self.log('Close, %.2f' % self.dataclose[0])
        
#         date = bt.num2date(self.date[0]).date()
#         prev_sentiment = self.sentiment
#         if date in date_sentiment:
#             self.sentiment = date_sentiment[date]
        
#         # Check if an order is pending. if yes, we cannot send a 2nd one
#         if self.order:
#             return
#         print(self.sentiment)
#         # If not in the market and previous sentiment not none
#         if not self.position and prev_sentiment:
#             # buy if current close more than sma AND sentiment increased by >= 0.5
#             if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
#                 self.log('BUY CREATE, %.2f' % self.dataclose[0])
#                 self.order = self.buy()
                
#         # Already in the market and previous sentiment not none
#         elif prev_sentiment:
#             # sell if current close less than sma AND sentiment decreased by >= 0.5
#             if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
#                 self.log('SELL CREATE, %.2f' % self.dataclose[0])
#                 self.order = self.sell()

#     def stop(self):
#         self.log('(MA Period %2d) Ending Value %.2f' %
#                  (self.params.period, self.broker.getvalue()), doprint=True)
        

# if __name__ == '__main__':
#     cerebro = bt.Cerebro()
    
#     # Strategy
#     cerebro.addstrategy(SentimentStrat)

#     # Data Feed
#     data = bt.feeds.YahooFinanceData(
#         dataname = 'FB',
#         fromdate = earliest_date,
#         todate = datetime.datetime(2018,11,25),
#         reverse = False
#     )
    
#     cerebro.adddata(data)

#     cerebro.broker.setcash(100000.0)
#     cerebro.addsizer(bt.sizers.FixedSize, stake=10)
#     cerebro.broker.setcommission(commission=0.001)
#     print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
#     cerebro.run()
#     print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
#     cerebro.plot()

In [2]:
# download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/alexbeckham/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# load API Key and initialize NewsAPI Client
api_key = os.getenv("news_api")

newsapi = NewsApiClient(api_key='eaef1a3e3a2645d6b7e1378d1ba9de43')

In [14]:
# get walmart returns for last 5 years 
from pathlib import Path

csvpath = Path('stock_data/WMT.csv')
wmt_df = pd.read_csv(csvpath, parse_dates=True, index_col='Date')

# Display data
wmt_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-04-12,67.570000,68.870003,67.449997,68.800003,61.571712,7734600
2016-04-13,69.010002,69.419998,68.870003,69.150002,61.884914,6262700
2016-04-14,69.150002,69.239998,68.150002,68.800003,61.571712,6977100
2016-04-15,68.940002,69.410004,68.940002,69.059998,61.804375,7584500
2016-04-18,69.050003,69.900002,68.849998,69.860001,62.520336,6622900


In [ ]:
# # Drop Outer Table Level
# # WMT_df = WMT_df.droplevel(axis=1, level=0)

# # Use the drop function to drop extra columns
# WMT_df = WMT_df.drop(columns=["open", "high", "low", "volume"], axis = 1)

# # Since this is daily data, we can keep only the date (remove the time) component of the data
# WMT_df.index = WMT_df.index.date

# # Display sample data
# WMT_df.head()

In [15]:
# fetch all news about WMT from NewsAPI
wmt_headlines = newsapi.get_everything(
    q="walmart",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# print total articles
print(f"Total articles about Walmart: {wmt_headlines['totalResults']}")

# show sample article
wmt_headlines["articles"][0]

Total articles about Walmart: 5852


{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
 'author': 'Manish Singh',
 'title': 'Flipkart to acquire online travel firm Cleartrip',
 'description': 'Flipkart said on Thursday it has agreed to acquire online travel firm Cleartrip as the Walmart-owned e-commerce firm looks to expand its offerings in the world’s second largest internet market. The deal values Cleartrip, which raised about $74 million prior t…',
 'url': 'http://techcrunch.com/2021/04/15/flipkart-to-acquire-online-travel-firm-cleartrip/',
 'urlToImage': 'https://techcrunch.com/wp-content/uploads/2019/07/flipkart_techcrunch.jpg?w=594',
 'publishedAt': '2021-04-15T08:39:31Z',
 'content': 'Flipkart said on Thursday it has agreed to acquire online travel firm Cleartrip as the Walmart-owned e-commerce firm looks to expand its offerings in the world’s second largest internet market.\r\nThe … [+1945 chars]'}

In [ ]:
# #transform the news dictionary to a dataframe
# wmt_df = pd.DataFrame(walmart_headlines["articles"])
# wmt_df.head()

In [16]:
# # Sentiment calculation based on compound score
# def get_sentiment(score):

#     result = 0  # Neutral by default
#     if score >= 0.05:  # Positive
#         result = 1
#     elif score <= -0.05:  # Negative
#         result = -1

#     return result

In [ ]:
# # Sentiment scores dictionaries
# title_sent = {
#     "title_compound": [],
#     "title_pos": [],
#     "title_neu": [],
#     "title_neg": [],
#     "title_sent": [],
# }
# text_sent = {
#     "text_compound": [],
#     "text_pos": [],
#     "text_neu": [],
#     "text_neg": [],
#     "text_sent": [],
# }

# # Get sentiment for the text and the title
# for index, row in wmt_df.iterrows():
#     try:
#         # Sentiment scoring with VADER
#         title_sentiment = analyzer.polarity_scores(row["title"])
#         title_sent["title_compound"].append(title_sentiment["compound"])
#         title_sent["title_pos"].append(title_sentiment["pos"])
#         title_sent["title_neu"].append(title_sentiment["neu"])
#         title_sent["title_neg"].append(title_sentiment["neg"])
#         title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))

#         text_sentiment = analyzer.polarity_scores(row["content"])
#         text_sent["text_compound"].append(text_sentiment["compound"])
#         text_sent["text_pos"].append(text_sentiment["pos"])
#         text_sent["text_neu"].append(text_sentiment["neu"])
#         text_sent["text_neg"].append(text_sentiment["neg"])
#         text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
#     except AttributeError:
#         pass

# # Attaching sentiment columns to the News DataFrame
# title_sentiment_df = pd.DataFrame(title_sent)
# text_sentiment_df = pd.DataFrame(text_sent)
# wmt_df = wmt_df.join(title_sentiment_df).join(text_sentiment_df)

# wmt_df.head()

In [ ]:

# # get sentiment of single article

# print(walmart_headlines['articles'][1]['description'])

# # view the score of the article's description
# analyzer.polarity_scores(walmart_headlines['articles'][1]['description'])

In [18]:
# get sentiment of all news articles

wmt_sentiments = []

for article in wmt_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"]#[:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"] 
        neu = sentiment["neu"]
        neg = sentiment["neg"] 
        
        wmt_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
wmt_sentiment_df = pd.DataFrame(wmt_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
wmt_sentiment_df = wmt_sentiment_df[cols]
wmt_sentiment_df

,date,text,compound,positive,negative,neutral
0,2021-04-15T08:39:31Z,Flipkart said on Thursday it has agreed to acq...,0.5267,0.124,0.000,0.876
1,2021-04-07T14:01:12Z,Thanks to major players like Amazon and Walmar...,0.8442,0.258,0.000,0.742
2,2021-03-22T15:34:10Z,We can probably all agree that its been a very...,0.3612,0.062,0.000,0.938
3,2021-04-15T18:19:02Z,5 with 4 posters participating\r\nWalmart is i...,0.0000,0.000,0.000,1.000
4,2021-04-12T22:54:00Z,Zest Labs Inc won $115 million from Walmart In...,0.7096,0.205,0.066,0.729
...,...,...,...,...,...,...
95,2021-04-05T17:59:34Z,"One year ago, Sam Dennigan got his plant-based...",-0.5551,0.000,0.125,0.875
96,2021-03-27T23:04:00Z,Even though Sony's new PlayStation 5 has been ...,-0.2732,0.000,0.055,0.945
97,2021-04-09T13:04:00Z,"Since its launch in November, it's been nearly...",0.8126,0.208,0.000,0.792
98,2021-04-07T01:35:00Z,The PlayStation 5 has a few big titles coming ...,-0.2263,0.000,0.049,0.951


In [11]:
# get the desriptive statistics
wmt_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.223806,0.082130,0.027410,0.890480
std,0.438641,0.090168,0.048987,0.096302
min,-0.784500,0.000000,0.000000,0.607000
25%,0.000000,0.000000,0.000000,0.819250
50%,0.140300,0.061500,0.000000,0.914500
75%,0.542300,0.131000,0.046750,0.968000
max,0.911800,0.324000,0.194000,1.000000
